# Use a brain atlas - visualize data in reference space

Here we use **brainrender**([code](https://github.com/brainglobe/brainrender), [docs](https://docs.brainrender.info), [preprint](https://www.biorxiv.org/content/10.1101/2020.02.23.961748v2)) to visualize our morphed ROIs in a common reference space.


before starting, make sure you
```
pip install brainrender
```
When running a brainrender visualization, a new window will open; to interrupt its execution, you can press esc, ctrl-C or Q (not necessarily all work :) )

Brainrender is thought to be used in scripts, more then notebooks. It should be fine in notebooks also, but there is a non-zero chance you won't be able to see a window opening, or to quit it. If it happens just move the code to a script.py file and run it from the terminal, after activating your environment, with 

```
python all_the/path/to/script.py
```

In [ ]:
 import numpy as np

from brainrender import Scene
from brainrender.actors import Points

from pathlib import Path

In [ ]:
# Those are necessary for running brainrender in a notebook - remove if using in scripts
from vedo import embedWindow                                                                
embedWindow(None)

# Load cells from previous notebook:
anatomies_folder = Path(r"/...")
cells = np.load(anatomies_folder / "morphed_coords1.npy")


# we did the morphing in a 3um downsampled space, but here we need to multiply them back in the original atlas space of 1um voxels
#normally this ugly arbitrary step does not happen if you morph, with more time, to the original atlas space
cells = cells * 3

# Create a brainrender scene using the zebrafish atlas:
scene = Scene(atlas_name="mpin_zfish_1um", title="Labelled cells")

# Add to scene - we use a subset of cells just for clarity:
scene.add(Points(cells[::10, :], name="CELLS", colors="steelblue", radius=2))

# render - click esc to interrupt!
scene.content
scene.render()

We can also add brain regions to the scene! For example:

In [ ]:
?scene.add_brain_region

In [ ]:
anatomies_folder = Path(r"/Users/luigipetrucco/Desktop/anatomies_orig")
cells = np.load(anatomies_folder / "morphed_coords1.npy")


# we did the morphing in a 3um downsampled space, but here we need to multiply them back in the original atlas space of 1um voxels
#normally this ugly arbitrary step does not happen if you morph, with more time, to the original atlas space
cells = cells * 3

# Create a brainrender scene using the zebrafish atlas:
scene = Scene(atlas_name="mpin_zfish_1um", title="Labelled cells")

# Add to scene - we use a subset of cells just for clarity:
scene.add(Points(cells[::10, :], name="CELLS", colors="steelblue", radius=2))

scene.add_brain_region("nucleus of the medial longitudinal fascicle", color="green", silhouette=True)

# render - click esc to interrupt!
scene.content
scene.render()

Brainrender relies on the bg-atlasapi library that we introduced in the previous notebook.
To explore the available regions you can look into:

In [ ]:
import numpy as np
from bg_atlasapi.bg_atlas import BrainGlobeAtlas

atlas = BrainGlobeAtlas("mpin_zfish_1um")
atlas.structures

Now try to play with the visualization, adding ROIs from all fish in different colors, or adding other brain regions!

# Visualizing brain regions in napari

We can also do a similar visualization in napari. The atlas class has also an "annotation" stack where each voxel has an index that corresponds to a brain region - at the finest level of the hierarchy we plotted above. To get composed regions, there is a "get_structure_mask" method. For a broader overview, you can have a look at the package [repo](https://github.com/brainglobe/bg-atlasapi) and [docs](https://docs.brainglobe.info/bg-atlasapi/usage)!

In [ ]:
ipn_mask = atlas.annotation == atlas.structures["nucleus of the medial longitudinal fascicle"]["id"]
cerebellum_mask = atlas.get_structure_mask("habenula")

In [ ]:
import napari
%gui qt

In [ ]:
# Open the viewer. Explore it both in slice view and in 3D view!

v = napari.view_image(atlas.reference)
v.add_labels(ipn_mask)
v.add_labels(cerebellum_mask)
v.add_points(cells[::10, :], n_dimensional=True, size=2)